# QA with reference

We believe that hallucinations pose a major problem in the adoption of LLMs (Language Model Models). It is imperative to provide a simple and quick solution that allows the user to verify the coherence of the answers to the questions they are asked.

The conventional approach is to provide a list of URLs of the documents that helped in answering (see qa_with_source). However, this approach is unsatisfactory in several scenarios:

1. The question is asked about a PDF of over 100 pages. Each fragment comes from the same document, but from where?
2. Some documents do not have URLs (data retrieved from a database or other loaders).

It appears essential to have a means of retrieving all references to the actual data sources used by the model to answer the question. 

This includes:
- The precise list of documents used for the answer (the `Documents`, along with their metadata that may contain page numbers, slide numbers, or any other information allowing the retrieval of the fragment in the original document).
- The excerpts of text used for the answer in each fragment. Even if a fragment is used, the LLM only utilizes a small portion to generate the answer. Access to these verbatim excerpts helps to quickly ascertain the validity of the answer.

We propose a new pipeline: `qa_with_reference` for this purpose. It is a Question/Answer type pipeline that returns the list of documents used, and in the metadata, the list of verbatim excerpts exploited to produce the answer.

*At this time, only the `map_reduce` chain type car extract the verbatim excerpts.*


In [1]:
#!pip install langchain-qa_with_references

In [9]:
from langchain import OpenAI
from langchain.schema import Document
llm = OpenAI(
            temperature=0,
        )

In [10]:

from langchain_qa_with_references.chains import QAWithReferencesAndVerbatimsChain
chain_type="map_reduce"  # Only map_reduce can extract the verbatim.
qa_chain = QAWithReferencesAndVerbatimsChain.from_chain_type(
        llm=llm,
        chain_type=chain_type,
    )

question = "what does it eat?"
bodies = ["he eats apples and plays football."
          "My name is Philippe."
          
          "he eats pears.",
          
          "he eats carrots. I like football.",
          "The Earth is round."
]
docs=[Document(page_content=body,metadata={"id":i}) for i,body in enumerate(bodies)]

answer = qa_chain(
        inputs={
            "docs": docs,
            "question": question,
        },
    )


print(f'To answer "{answer["answer"]}", the LLM use:')
for doc in answer["source_documents"]:
    print(f"Document {doc.metadata['id']}")
    for verbatim in doc.metadata.get("verbatims",[]):
        print(f'- "{verbatim}"')


To answer "", the LLM use:
Document 0
- "eats apples"
- "eats pears."
Document 1
- "he eats carrots."


In [23]:
#!pip install faiss-cpu sentence_transformers

In [24]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('./', glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()

# Text Splitters
from langchain.text_splitter import MarkdownTextSplitter
markdown_splitter = MarkdownTextSplitter(chunk_size=2000, chunk_overlap=100, length_function=len)
md_docs = markdown_splitter.split_documents(docs)

# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings  # create custom embeddings class that just calls API
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vector stores (pip install faiss or pip install faiss-cpu)
from langchain.vectorstores import FAISS
db = FAISS.from_documents(md_docs, embeddings)

# Retrievers
retriever = db.as_retriever(search_kwargs={"k": 4})
question = "How do I use OpenAI?"

In [22]:
from langchain_qa_with_references.chains import RetrievalQAWithReferencesAndVerbatimsChain
from typing import Literal
chain_type:Literal["stuff","map_reduce","map_rerank","refine"]="map_reduce"

qa_chain = RetrievalQAWithReferencesAndVerbatimsChain.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    reduce_k_below_max_tokens=True,
)
answer = qa_chain(
    inputs={
        "question": question,
    }
)
print(f'For the question "{question}", to answer "{answer["answer"]}", the LLM use:')
for doc in answer["source_documents"]:
    print(f"Source {doc.metadata['source']}")
    for verbatim in doc.metadata.get("verbatims", []):
        print(f'-  "{verbatim}"')

For the question "How do I use OpenAI?", to answer "", the LLM use:
Source ./_dist/docs_skeleton/docs/integrations/chat/openai.md
-  "This notebook covers how to get started with OpenAI chat models."
Source ./_dist/docs_skeleton/docs/modules/agents/how_to/use_toolkits_with_openai_functions.md
-  "This notebook shows how to use the OpenAI functions agent with arbitrary toolkits."
Source ./_dist/docs_skeleton/docs/guides/adapters/openai.md
-  "At the moment this only deals with output and does not return other information (token counts, stop reasons, etc"
-  "LangChain's integrations with many model providers make this easy to do so."
-  "While LangChain has it's own message and model APIs, we've also made it as easy as possible to explore other models by exposing an adapter to adapt LangChain models to the OpenAI api."
